In [ ]:
import os
import logging
import joblib
import uvicorn
import nest_asyncio
import numpy as np
from fastapi import FastAPI
from typing import List
from pydantic import BaseModel

#  Asegurar que la carpeta 'logs/' existe
if not os.path.exists("logs"):
    os.makedirs("logs")

#  Configurar ruta absoluta para el archivo de logs
log_file = os.path.join(os.getcwd(), "logs", "app.log")

# Configuración de logging con ruta absoluta
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file, mode="a"),  # Guardar logs en archivo
        logging.StreamHandler()  # Mostrar logs en consola
    ],
    force=True
)

logger = logging.getLogger(__name__)

# Inicializar FastAPI
app = FastAPI()

#  Cargar modelo
try:
    model = joblib.load("modelo_cancer.pkl")
    logger.info("Modelo cargado exitosamente.")
except Exception as e:
    logger.error(f"Error cargando el modelo: {e}")
    model = None

#🔹 Definir la estructura de entrada
class InputData(BaseModel):
    features: List[float]

@app.get("/")
def read_root():
    logger.info("Solicitud recibida en '/'")
    return {"message": "API en funcionamiento"}

@app.post("/predict")
def predict(data: InputData):
    if model is None:
        logger.error("Modelo no disponible")
        return {"error": "Modelo no disponible"}
    
    try:
        features_array = np.array(data.features).reshape(1, -1)
        prediction = model.predict(features_array)
        logger.info(f"Predicción realizada: {prediction.tolist()}")
        return {"prediction": prediction.tolist()}
    except Exception as e:
        logger.error(f"Error en la predicción: {e}")
        return {"error": "Error procesando la predicción"}

#  Permitir ejecución en Jupyter Notebook (si aplica)
nest_asyncio.apply()

#  Iniciar servidor FastAPI
if __name__ == "__main__":
    logger.info("Iniciando servidor FastAPI en http://0.0.0.0:8000")
    logger.info("Prueba de log: La aplicación está iniciando.")
    uvicorn.run(app, host="0.0.0.0", port=8000)



2025-03-27 14:59:16,989 - INFO - Modelo cargado exitosamente.
2025-03-27 14:59:16,995 - INFO - Iniciando servidor FastAPI en http://0.0.0.0:8000
2025-03-27 14:59:16,996 - INFO - Prueba de log: La aplicación está iniciando.
INFO:     Started server process [19076]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
2025-03-27 14:59:34,729 - INFO - Predicción realizada: [1]


INFO:     127.0.0.1:64538 - "POST /predict HTTP/1.1" 200 OK
INFO:     186.84.89.128:0 - "POST /content HTTP/1.1" 404 Not Found


2025-03-27 15:16:11,583 - INFO - Predicción realizada: [1]


INFO:     186.84.89.128:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [19076]
